In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd

import unicodedata
import re
import json

import acquire

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filter="ignore"

from bs4 import BeautifulSoup
import prepare
import plotly.express as px

***

# `Acquire` Data.

In [2]:
df = acquire.get_fifa_data()

# `Explore` And `Prep` Data

In [3]:
#forces dataframe to show every columns
pd.set_option('display.max_columns', None)
df

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",94,94,77000000.0,400000.0,33,1985-02-05,187,83,45.0,Juventus,Italian Serie A,1.0,ST,7.0,NaN,2018-07-10,2022.0,38,Portugal,1354.0,LS,7.0,Right,4,5,5,High/Low,Unique,Yes,127100000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Diver, Flair, Long Shot Taker...",90.0,93.0,81.0,89.0,35.0,79.0,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,NaN,92+2,92+2,92+2,90,91,91,91,90,88+4,88+4,88+4,88+4,81+4,81+4,81+4,88+4,65+4,61+4,61+4,61+4,65+4,61+4,54+4,54+4,54+4,61+4,20+4,https://cdn.sofifa.net/players/020/801/19_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,2019
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"CF, RW, ST",94,94,110500000.0,575000.0,31,1987-06-24,170,72,241.0,FC Barcelona,Spain Primera Division,1.0,RW,10.0,NaN,2004-07-01,2021.0,52,Argentina,1369.0,CF,10.0,Left,4,4,5,Medium/Medium,Unique,Yes,226500000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",88.0,91.0,88.0,96.0,32.0,61.0,77,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,NaN,89+5,89+5,89+5,92+2,93+1,93+1,93+1,92+2,93+1,93+1,93+1,91+3,85+7,85+7,85+7,91+3,63+7,61+7,61+7,61+7,63+7,59+7,48+7,48+7,48+7,59+7,19+7,https://cdn.sofifa.net/players/158/023/19_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar,Neymar da Silva Santos Júnior,LW,92,93,118500000.0,300000.0,26,1992-02-05,175,68,73.0,Paris Saint-Germain,French Ligue 1,1.0,LW,10.0,NaN,2017-08-03,2022.0,54,Brazil,1370.0,LW,10.0,Right,5,5,5,High/Medium,Unique,Yes,228100000.0,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Diver, Selfish, Flair, Speed Dribbler (AI), Te...",92.0,84.0,83.0,95.0,32.0,59.0,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94,27,24,33,9,9,15,15,11,NaN,84+7,84+7,84+7,90+3,90+3,90+3,90+3,90+3,90+3,90+3,90+3,89+4,81+7,81+7,81+7,89+4,66+7,61+7,61+7,61+7,66+7,61+7,47+7,47+7,47+7,61+7,20+7,https://cdn.sofifa.net/players/190/871/19_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1370/60.png,https:

In [4]:
#add 0 values to non goal keeper players
#df['goalkeeping_speed'].fillna("0", inplace = True)

In [5]:
#forces dataframe to show every columns
pd.set_option('display.max_columns', None)

***

# Columns That Have Null Values

In [6]:
na = df.isna().sum() > 0
na[na]

value_eur                    True
wage_eur                     True
club_team_id                 True
club_name                    True
league_name                  True
league_level                 True
club_position                True
club_jersey_number           True
club_loaned_from             True
club_joined                  True
club_contract_valid_until    True
nation_team_id               True
nation_position              True
nation_jersey_number         True
release_clause_eur           True
player_tags                  True
player_traits                True
pace                         True
shooting                     True
passing                      True
dribbling                    True
defending                    True
physic                       True
goalkeeping_speed            True
club_logo_url                True
club_flag_url                True
nation_logo_url              True
dtype: bool

`Takeaways:`
- Over 25 columns have null values and we need to address them.

***

## Taking a look at our null values:

# `value_eur`,`wage_eur`, & `club_team_id`.

In [7]:
df[df.value_eur.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
422,156713,https://sofifa.com/player/156713/andreas-granq...,A. Granqvist,Andreas Granqvist,CB,80,80,NaN,NaN,33,1985-04-16,192,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,Sweden,1363.0,LCB,4.0,Right,4,2,2,High/Medium,Normal (185+),No,NaN,NaN,"Leadership, Team Player",48.0,58.0,59.0,55.0,82.0,78.0,49,51,81,73,37,49,36,40,67,63,46,49,55,76,36,74,64,67,83,59,81,82,54,49,79,78,82,83,79,7,9,12,10,15,NaN,62+1,62+1,62+1,56,59,59,59,56,59+1,59+1,59+1,57+1,65+1,65+1,65+1,57+1,69+1,75+1,75+1,75+1,69+1,71+1,79+1,79+1,79+1,71+1,18+1,https://cdn.sofifa.net/players/156/713/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,2019
507,207439,https://sofifa.com/player/207439/leandro-pared...,L. Paredes,Leandro Daniel Paredes,"CM, CDM",80,85,NaN,NaN,24,1994-06-29,180,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,Argentina,1369.0,SUB,5.0,Right,4,4,2,Medium/Medium,Normal (170-185),No,NaN,NaN,"Power Free-Kick, Flair, Long Passer (AI), Long...",72.0,67.0,81.0,79.0,72.0,73.0,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,NaN,71+1,71+1,71+1,76,75,75,75,76,78+1,78+1,78+1,77+1,79+1,79+1,79+1,77+1,76+1,77+1,77+1,77+1,76+1,74+1,72+1,72+1,72+1,74+1,17+1,https://cdn.sofifa.net/players/207/439/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
592,187347,https://sofifa.com/player/187347/igor-smolniko...,I. Smolnikov,Igor Smolnikov,RB,79,79,NaN,NaN,29,1988-08-08,178,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,SUB,2.0,Right,3,3,2,High/High,Lean (170-185),No,NaN,#Engine,"Injury Prone, Dives Into Tackles (AI), Speed D...",87.0,61.0,73.0,73.0,75.0,75.0,73,61,69,79,57,72,49,46,75,72,84,90,80,75,76,67,85,93,68,57,65,71,77,72,41,73,76,76,80,7,12,10,8,15,NaN,71+1,71+1,71+1,74,73,73,73,74,74+1,74+1,74+1,76+1,75+1,75+1,75+1,76+1,78+1,75+1,75+1,75+1,78+1,78+1,74+1,74+1,74+1,78+1,18+1,https://cdn.sofifa.net/players/187/347/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2019
593,187457,https://sofifa.com/player/187457/christian-nob...,C. Noboa,Christian Fernando Noboa Tello,"CM, CDM",79,79,NaN,NaN,33,1985-04-09,183,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,Ecuador,111465.0,RCM,18.0,Right,3,4,2,High/High,Normal (170-185),No,NaN,#Engine,"Long Passer (AI), Long Shot Taker (AI), Playma...",73.0,71.0,78.0,77.0,75.0,76.0,74,65,72,78,66,77,76,81,80,80,73,73,77,75,66,75,79,90,71,79,68,78,72,78,80,72,75,75,75,15,8,11,15,11,NaN,73+1,73+1,73+1,75,75,75,75,75,76+1,76+1,76+1,76+1,78+1,78+1,78+1,76+1,77+1,77+1,77+1,77

`Takeaways:`
- we might have to consider removing these players because they would corrupt our data considering their wage information is missing and would not add any value to our model.

In [8]:
df[df.wage_eur.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
422,156713,https://sofifa.com/player/156713/andreas-granq...,A. Granqvist,Andreas Granqvist,CB,80,80,NaN,NaN,33,1985-04-16,192,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,Sweden,1363.0,LCB,4.0,Right,4,2,2,High/Medium,Normal (185+),No,NaN,NaN,"Leadership, Team Player",48.0,58.0,59.0,55.0,82.0,78.0,49,51,81,73,37,49,36,40,67,63,46,49,55,76,36,74,64,67,83,59,81,82,54,49,79,78,82,83,79,7,9,12,10,15,NaN,62+1,62+1,62+1,56,59,59,59,56,59+1,59+1,59+1,57+1,65+1,65+1,65+1,57+1,69+1,75+1,75+1,75+1,69+1,71+1,79+1,79+1,79+1,71+1,18+1,https://cdn.sofifa.net/players/156/713/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,2019
507,207439,https://sofifa.com/player/207439/leandro-pared...,L. Paredes,Leandro Daniel Paredes,"CM, CDM",80,85,NaN,NaN,24,1994-06-29,180,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,Argentina,1369.0,SUB,5.0,Right,4,4,2,Medium/Medium,Normal (170-185),No,NaN,NaN,"Power Free-Kick, Flair, Long Passer (AI), Long...",72.0,67.0,81.0,79.0,72.0,73.0,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,NaN,71+1,71+1,71+1,76,75,75,75,76,78+1,78+1,78+1,77+1,79+1,79+1,79+1,77+1,76+1,77+1,77+1,77+1,76+1,74+1,72+1,72+1,72+1,74+1,17+1,https://cdn.sofifa.net/players/207/439/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
592,187347,https://sofifa.com/player/187347/igor-smolniko...,I. Smolnikov,Igor Smolnikov,RB,79,79,NaN,NaN,29,1988-08-08,178,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,SUB,2.0,Right,3,3,2,High/High,Lean (170-185),No,NaN,#Engine,"Injury Prone, Dives Into Tackles (AI), Speed D...",87.0,61.0,73.0,73.0,75.0,75.0,73,61,69,79,57,72,49,46,75,72,84,90,80,75,76,67,85,93,68,57,65,71,77,72,41,73,76,76,80,7,12,10,8,15,NaN,71+1,71+1,71+1,74,73,73,73,74,74+1,74+1,74+1,76+1,75+1,75+1,75+1,76+1,78+1,75+1,75+1,75+1,78+1,78+1,74+1,74+1,74+1,78+1,18+1,https://cdn.sofifa.net/players/187/347/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2019
593,187457,https://sofifa.com/player/187457/christian-nob...,C. Noboa,Christian Fernando Noboa Tello,"CM, CDM",79,79,NaN,NaN,33,1985-04-09,183,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,Ecuador,111465.0,RCM,18.0,Right,3,4,2,High/High,Normal (170-185),No,NaN,#Engine,"Long Passer (AI), Long Shot Taker (AI), Playma...",73.0,71.0,78.0,77.0,75.0,76.0,74,65,72,78,66,77,76,81,80,80,73,73,77,75,66,75,79,90,71,79,68,78,72,78,80,72,75,75,75,15,8,11,15,11,NaN,73+1,73+1,73+1,75,75,75,75,75,76+1,76+1,76+1,76+1,78+1,78+1,78+1,76+1,77+1,77+1,77+1,77

`Takeaways:`
- we might have to consider removing these players because they would corrupt our data considering their value information is missing and would not add any value to our model.

In [9]:
df[df.club_team_id.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
422,156713,https://sofifa.com/player/156713/andreas-granq...,A. Granqvist,Andreas Granqvist,CB,80,80,NaN,NaN,33,1985-04-16,192,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,Sweden,1363.0,LCB,4.0,Right,4,2,2,High/Medium,Normal (185+),No,NaN,NaN,"Leadership, Team Player",48.0,58.0,59.0,55.0,82.0,78.0,49,51,81,73,37,49,36,40,67,63,46,49,55,76,36,74,64,67,83,59,81,82,54,49,79,78,82,83,79,7,9,12,10,15,NaN,62+1,62+1,62+1,56,59,59,59,56,59+1,59+1,59+1,57+1,65+1,65+1,65+1,57+1,69+1,75+1,75+1,75+1,69+1,71+1,79+1,79+1,79+1,71+1,18+1,https://cdn.sofifa.net/players/156/713/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,2019
507,207439,https://sofifa.com/player/207439/leandro-pared...,L. Paredes,Leandro Daniel Paredes,"CM, CDM",80,85,NaN,NaN,24,1994-06-29,180,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,Argentina,1369.0,SUB,5.0,Right,4,4,2,Medium/Medium,Normal (170-185),No,NaN,NaN,"Power Free-Kick, Flair, Long Passer (AI), Long...",72.0,67.0,81.0,79.0,72.0,73.0,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,NaN,71+1,71+1,71+1,76,75,75,75,76,78+1,78+1,78+1,77+1,79+1,79+1,79+1,77+1,76+1,77+1,77+1,77+1,76+1,74+1,72+1,72+1,72+1,74+1,17+1,https://cdn.sofifa.net/players/207/439/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
592,187347,https://sofifa.com/player/187347/igor-smolniko...,I. Smolnikov,Igor Smolnikov,RB,79,79,NaN,NaN,29,1988-08-08,178,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,SUB,2.0,Right,3,3,2,High/High,Lean (170-185),No,NaN,#Engine,"Injury Prone, Dives Into Tackles (AI), Speed D...",87.0,61.0,73.0,73.0,75.0,75.0,73,61,69,79,57,72,49,46,75,72,84,90,80,75,76,67,85,93,68,57,65,71,77,72,41,73,76,76,80,7,12,10,8,15,NaN,71+1,71+1,71+1,74,73,73,73,74,74+1,74+1,74+1,76+1,75+1,75+1,75+1,76+1,78+1,75+1,75+1,75+1,78+1,78+1,74+1,74+1,74+1,78+1,18+1,https://cdn.sofifa.net/players/187/347/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2019
593,187457,https://sofifa.com/player/187457/christian-nob...,C. Noboa,Christian Fernando Noboa Tello,"CM, CDM",79,79,NaN,NaN,33,1985-04-09,183,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,Ecuador,111465.0,RCM,18.0,Right,3,4,2,High/High,Normal (170-185),No,NaN,#Engine,"Long Passer (AI), Long Shot Taker (AI), Playma...",73.0,71.0,78.0,77.0,75.0,76.0,74,65,72,78,66,77,76,81,80,80,73,73,77,75,66,75,79,90,71,79,68,78,72,78,80,72,75,75,75,15,8,11,15,11,NaN,73+1,73+1,73+1,75,75,75,75,75,76+1,76+1,76+1,76+1,78+1,78+1,78+1,76+1,77+1,77+1,77+1,77

`Takeaways:`
- we might have to consider removing these players because there club id is important information that we need to conclude their club budget.

In [10]:
df[df.league_name.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
422,156713,https://sofifa.com/player/156713/andreas-granq...,A. Granqvist,Andreas Granqvist,CB,80,80,NaN,NaN,33,1985-04-16,192,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,Sweden,1363.0,LCB,4.0,Right,4,2,2,High/Medium,Normal (185+),No,NaN,NaN,"Leadership, Team Player",48.0,58.0,59.0,55.0,82.0,78.0,49,51,81,73,37,49,36,40,67,63,46,49,55,76,36,74,64,67,83,59,81,82,54,49,79,78,82,83,79,7,9,12,10,15,NaN,62+1,62+1,62+1,56,59,59,59,56,59+1,59+1,59+1,57+1,65+1,65+1,65+1,57+1,69+1,75+1,75+1,75+1,69+1,71+1,79+1,79+1,79+1,71+1,18+1,https://cdn.sofifa.net/players/156/713/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,2019
507,207439,https://sofifa.com/player/207439/leandro-pared...,L. Paredes,Leandro Daniel Paredes,"CM, CDM",80,85,NaN,NaN,24,1994-06-29,180,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,Argentina,1369.0,SUB,5.0,Right,4,4,2,Medium/Medium,Normal (170-185),No,NaN,NaN,"Power Free-Kick, Flair, Long Passer (AI), Long...",72.0,67.0,81.0,79.0,72.0,73.0,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,NaN,71+1,71+1,71+1,76,75,75,75,76,78+1,78+1,78+1,77+1,79+1,79+1,79+1,77+1,76+1,77+1,77+1,77+1,76+1,74+1,72+1,72+1,72+1,74+1,17+1,https://cdn.sofifa.net/players/207/439/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
592,187347,https://sofifa.com/player/187347/igor-smolniko...,I. Smolnikov,Igor Smolnikov,RB,79,79,NaN,NaN,29,1988-08-08,178,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,SUB,2.0,Right,3,3,2,High/High,Lean (170-185),No,NaN,#Engine,"Injury Prone, Dives Into Tackles (AI), Speed D...",87.0,61.0,73.0,73.0,75.0,75.0,73,61,69,79,57,72,49,46,75,72,84,90,80,75,76,67,85,93,68,57,65,71,77,72,41,73,76,76,80,7,12,10,8,15,NaN,71+1,71+1,71+1,74,73,73,73,74,74+1,74+1,74+1,76+1,75+1,75+1,75+1,76+1,78+1,75+1,75+1,75+1,78+1,78+1,74+1,74+1,74+1,78+1,18+1,https://cdn.sofifa.net/players/187/347/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2019
593,187457,https://sofifa.com/player/187457/christian-nob...,C. Noboa,Christian Fernando Noboa Tello,"CM, CDM",79,79,NaN,NaN,33,1985-04-09,183,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,Ecuador,111465.0,RCM,18.0,Right,3,4,2,High/High,Normal (170-185),No,NaN,#Engine,"Long Passer (AI), Long Shot Taker (AI), Playma...",73.0,71.0,78.0,77.0,75.0,76.0,74,65,72,78,66,77,76,81,80,80,73,73,77,75,66,75,79,90,71,79,68,78,72,78,80,72,75,75,75,15,8,11,15,11,NaN,73+1,73+1,73+1,75,75,75,75,75,76+1,76+1,76+1,76+1,78+1,78+1,78+1,76+1,77+1,77+1,77+1,77

`Takeaways:`
- we might have to consider removing these players because the leage that they play in is important and can give us valuable information that needs to be categorized.

# `club_position`, `club_joined`, & `club_contract_valid_until`.

In [11]:
df[df.club_position.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
422,156713,https://sofifa.com/player/156713/andreas-granq...,A. Granqvist,Andreas Granqvist,CB,80,80,NaN,NaN,33,1985-04-16,192,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,Sweden,1363.0,LCB,4.0,Right,4,2,2,High/Medium,Normal (185+),No,NaN,NaN,"Leadership, Team Player",48.0,58.0,59.0,55.0,82.0,78.0,49,51,81,73,37,49,36,40,67,63,46,49,55,76,36,74,64,67,83,59,81,82,54,49,79,78,82,83,79,7,9,12,10,15,NaN,62+1,62+1,62+1,56,59,59,59,56,59+1,59+1,59+1,57+1,65+1,65+1,65+1,57+1,69+1,75+1,75+1,75+1,69+1,71+1,79+1,79+1,79+1,71+1,18+1,https://cdn.sofifa.net/players/156/713/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,2019
507,207439,https://sofifa.com/player/207439/leandro-pared...,L. Paredes,Leandro Daniel Paredes,"CM, CDM",80,85,NaN,NaN,24,1994-06-29,180,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,Argentina,1369.0,SUB,5.0,Right,4,4,2,Medium/Medium,Normal (170-185),No,NaN,NaN,"Power Free-Kick, Flair, Long Passer (AI), Long...",72.0,67.0,81.0,79.0,72.0,73.0,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,NaN,71+1,71+1,71+1,76,75,75,75,76,78+1,78+1,78+1,77+1,79+1,79+1,79+1,77+1,76+1,77+1,77+1,77+1,76+1,74+1,72+1,72+1,72+1,74+1,17+1,https://cdn.sofifa.net/players/207/439/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
592,187347,https://sofifa.com/player/187347/igor-smolniko...,I. Smolnikov,Igor Smolnikov,RB,79,79,NaN,NaN,29,1988-08-08,178,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,SUB,2.0,Right,3,3,2,High/High,Lean (170-185),No,NaN,#Engine,"Injury Prone, Dives Into Tackles (AI), Speed D...",87.0,61.0,73.0,73.0,75.0,75.0,73,61,69,79,57,72,49,46,75,72,84,90,80,75,76,67,85,93,68,57,65,71,77,72,41,73,76,76,80,7,12,10,8,15,NaN,71+1,71+1,71+1,74,73,73,73,74,74+1,74+1,74+1,76+1,75+1,75+1,75+1,76+1,78+1,75+1,75+1,75+1,78+1,78+1,74+1,74+1,74+1,78+1,18+1,https://cdn.sofifa.net/players/187/347/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2019
593,187457,https://sofifa.com/player/187457/christian-nob...,C. Noboa,Christian Fernando Noboa Tello,"CM, CDM",79,79,NaN,NaN,33,1985-04-09,183,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,Ecuador,111465.0,RCM,18.0,Right,3,4,2,High/High,Normal (170-185),No,NaN,#Engine,"Long Passer (AI), Long Shot Taker (AI), Playma...",73.0,71.0,78.0,77.0,75.0,76.0,74,65,72,78,66,77,76,81,80,80,73,73,77,75,66,75,79,90,71,79,68,78,72,78,80,72,75,75,75,15,8,11,15,11,NaN,73+1,73+1,73+1,75,75,75,75,75,76+1,76+1,76+1,76+1,78+1,78+1,78+1,76+1,77+1,77+1,77+1,77

In [12]:
df[df.club_joined.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
29,167664,https://sofifa.com/player/167664/gonzalo-higua...,G. Higuaín,Gonzalo Gerardo Higuaín,ST,88,88,57000000.0,225000.0,30,1987-12-10,186,89,47.0,AC Milan,Italian Serie A,1.0,ST,9.0,Juventus,NaN,2019.0,52,Argentina,1369.0,SUB,9.0,Right,4,3,4,High/Medium,Normal (170-185),Yes,NaN,#Clinical Finisher,"Outside Foot Shot, Through Ball",73.0,87.0,70.0,83.0,31.0,74.0,68,92,80,75,90,84,74,62,59,85,73,73,75,86,69,86,79,70,85,80,50,20,92,74,70,86,35,22,18,5,12,7,5,10,NaN,85+3,85+3,85+3,81,83,83,83,81,80+3,80+3,80+3,79+3,72+3,72+3,72+3,79+3,56+3,53+3,53+3,53+3,56+3,52+3,48+3,48+3,48+3,52+3,17+3,https://cdn.sofifa.net/players/167/664/19_120.png,https://cdn.sofifa.net/teams/47/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
38,198710,https://sofifa.com/player/198710/james-rodrigu...,J. Rodríguez,James David Rodríguez Rubio,"CAM, CM, RM",88,89,69500000.0,170000.0,26,1991-07-12,181,78,21.0,FC Bayern München,German 1. Bundesliga,1.0,LCM,11.0,Real Madrid CF,NaN,2019.0,56,Colombia,111109.0,LM,10.0,Left,3,4,4,Medium/Medium,Normal (170-185),Yes,NaN,"#Playmaker, #Distance Shooter, #Crosser, #FK S...","Avoids Using Weaker Foot, Flair, Playmaker (AI...",72.0,86.0,88.0,86.0,50.0,67.0,90,83,62,89,90,85,89,86,83,90,75,69,83,85,76,86,54,70,68,92,64,55,80,89,81,87,52,41,44,15,15,15,5,14,NaN,81+3,81+3,81+3,85,84,84,84,85,86+3,86+3,86+3,84+3,82+3,82+3,82+3,84+3,70+3,69+3,69+3,69+3,70+3,66+3,59+3,59+3,59+3,66+3,20+3,https://cdn.sofifa.net/players/198/710/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/111109/60.png,https://cdn.sofifa.net/flags/co.png,2019
95,187961,https://sofifa.com/player/187961/jose-paulo-be...,Paulinho,José Paulo Bezzera Maciel Júnior,"CM, CAM, CDM, RM",85,85,37000000.0,40000.0,29,1988-07-25,181,81,111839.0,Guangzhou FC,Chinese Super League,1.0,LDM,9.0,FC Barcelona,NaN,2019.0,54,Brazil,NaN,NaN,NaN,Right,3,3,3,High/High,Lean (170-185),Yes,NaN,#Engine,"Power Free-Kick, Dives Into Tackles (AI)",73.0,81.0,77.0,81.0,79.0,84.0,64,81,85,84,75,82,66,72,80,84,73,73,69,84,66,83,90,92,81,80,81,81,84,78,71,80,75,81,73,16,16,11,6,8,NaN,82+2,82+2,82+2,79,82,82,82,79,81+2,81+2,81+2,80+2,83+2,83+2,83+2,80+2,79+2,81+2,81+2,81+2,79+2,79+2,80+2,80+2,80+2,79+2,19+2,https://cdn.sofifa.net/players/187/961/19_120.png,https://cdn.sofifa.net/teams/111839/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/br.png,2019
189,207410,https://sofifa.com/player/207410/mateo-kovacic...,M. Kovačić,Mateo Kovačić,CM,83,89,35000000.0,12

In [13]:
df[df.club_contract_valid_until.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
422,156713,https://sofifa.com/player/156713/andreas-granq...,A. Granqvist,Andreas Granqvist,CB,80,80,NaN,NaN,33,1985-04-16,192,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,Sweden,1363.0,LCB,4.0,Right,4,2,2,High/Medium,Normal (185+),No,NaN,NaN,"Leadership, Team Player",48.0,58.0,59.0,55.0,82.0,78.0,49,51,81,73,37,49,36,40,67,63,46,49,55,76,36,74,64,67,83,59,81,82,54,49,79,78,82,83,79,7,9,12,10,15,NaN,62+1,62+1,62+1,56,59,59,59,56,59+1,59+1,59+1,57+1,65+1,65+1,65+1,57+1,69+1,75+1,75+1,75+1,69+1,71+1,79+1,79+1,79+1,71+1,18+1,https://cdn.sofifa.net/players/156/713/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,2019
507,207439,https://sofifa.com/player/207439/leandro-pared...,L. Paredes,Leandro Daniel Paredes,"CM, CDM",80,85,NaN,NaN,24,1994-06-29,180,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,Argentina,1369.0,SUB,5.0,Right,4,4,2,Medium/Medium,Normal (170-185),No,NaN,NaN,"Power Free-Kick, Flair, Long Passer (AI), Long...",72.0,67.0,81.0,79.0,72.0,73.0,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,NaN,71+1,71+1,71+1,76,75,75,75,76,78+1,78+1,78+1,77+1,79+1,79+1,79+1,77+1,76+1,77+1,77+1,77+1,76+1,74+1,72+1,72+1,72+1,74+1,17+1,https://cdn.sofifa.net/players/207/439/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2019
592,187347,https://sofifa.com/player/187347/igor-smolniko...,I. Smolnikov,Igor Smolnikov,RB,79,79,NaN,NaN,29,1988-08-08,178,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,SUB,2.0,Right,3,3,2,High/High,Lean (170-185),No,NaN,#Engine,"Injury Prone, Dives Into Tackles (AI), Speed D...",87.0,61.0,73.0,73.0,75.0,75.0,73,61,69,79,57,72,49,46,75,72,84,90,80,75,76,67,85,93,68,57,65,71,77,72,41,73,76,76,80,7,12,10,8,15,NaN,71+1,71+1,71+1,74,73,73,73,74,74+1,74+1,74+1,76+1,75+1,75+1,75+1,76+1,78+1,75+1,75+1,75+1,78+1,78+1,74+1,74+1,74+1,78+1,18+1,https://cdn.sofifa.net/players/187/347/19_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2019
593,187457,https://sofifa.com/player/187457/christian-nob...,C. Noboa,Christian Fernando Noboa Tello,"CM, CDM",79,79,NaN,NaN,33,1985-04-09,183,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,Ecuador,111465.0,RCM,18.0,Right,3,4,2,High/High,Normal (170-185),No,NaN,#Engine,"Long Passer (AI), Long Shot Taker (AI), Playma...",73.0,71.0,78.0,77.0,75.0,76.0,74,65,72,78,66,77,76,81,80,80,73,73,77,75,66,75,79,90,71,79,68,78,72,78,80,72,75,75,75,15,8,11,15,11,NaN,73+1,73+1,73+1,75,75,75,75,75,76+1,76+1,76+1,76+1,78+1,78+1,78+1,76+1,77+1,77+1,77+1,77

`takeaways:`
- Club_position, club_joined, & club_contract_valid_until all seem to have similar payers with null values.
- We will be dropping these rows to better benefit our exploration and our modeling.

# `nation_team_id`, `nation_position`.

In [14]:
df[df.nation_team_id.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,177003,https://sofifa.com/player/177003/luka-modric/1...,L. Modrić,Luka Modrić,"CM, CDM",91,91,67000000.0,425000.0,32,1985-09-09,172,66,243.0,Real Madrid CF,Spain Primera Division,1.0,RCM,10.0,NaN,2012-08-01,2020.0,10,Croatia,NaN,NaN,NaN,Right,4,4,4,High/High,Lean (170-185),Yes,137400000.0,"#Dribbler, #Playmaker, #Engine, #Crosser, #Acr...","Leadership, Long Passer (AI), Long Shot Taker ...",76.0,76.0,90.0,91.0,70.0,67.0,86,72,55,93,76,90,85,78,88,93,80,72,93,90,94,79,68,89,58,82,62,83,79,92,82,84,60,76,73,13,9,7,14,9,NaN,78+3,78+3,78+3,86,85,85,85,86,88+3,88+3,88+3,87+3,88+3,88+3,88+3,87+3,83+3,82+3,82+3,82+3,83+3,80+3,71+3,71+3,71+3,80+3,20+3,https://cdn.sofifa.net/players/177/003/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/hr.png,2019
17,168542,https://sofifa.com/player/168542/david-josue-j...,David Silva,David Josué Jiménez Silva,"CAM, CM",89,89,51500000.0,275000.0,32,1986-01-08,173,67,10.0,Manchester City,English Premier League,1.0,LCM,21.0,NaN,2010-07-14,2020.0,45,Spain,NaN,NaN,NaN,Left,2,4,4,High/Medium,Normal (170-185),Yes,95300000.0,"#Dribbler, #Playmaker, #Acrobat","Avoids Using Weaker Foot, Finesse Shot, Flair,...",66.0,74.0,87.0,90.0,50.0,60.0,83,73,54,92,80,89,82,76,81,93,69,64,92,87,90,71,64,78,52,73,56,48,88,92,75,92,57,50,29,6,15,7,6,12,NaN,77+3,77+3,77+3,84,84,84,84,84,87+2,87+2,87+2,84+3,84+3,84+3,84+3,84+3,68+3,69+3,69+3,69+3,68+3,64+3,56+3,56+3,56+3,64+3,18+3,https://cdn.sofifa.net/players/168/542/19_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/es.png,2019
26,1179,https://sofifa.com/player/1179/gianluigi-buffo...,G. Buffon,Gianluigi Buffon,GK,88,88,4000000.0,80000.0,40,1978-01-28,192,92,73.0,Paris Saint-Germain,French Ligue 1,1.0,GK,1.0,NaN,2018-07-06,2020.0,27,Italy,NaN,NaN,NaN,Right,2,1,4,Medium/Medium,Normal (185+),Yes,7400000.0,NaN,"Team Player, Cautious With Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,15,13,37,17,26,20,13,35,28,49,43,55,79,49,39,75,39,69,13,38,28,12,50,22,70,13,11,11,88,87,74,90,83,46.0,31+3,31+3,31+3,32,33,33,33,32,35+3,35+3,35+3,34+3,35+3,35+3,35+3,34+3,30+3,33+3,33+3,33+3,30+3,28+3,29+3,29+3,29+3,28+3,85+3,https://cdn.sofifa.net/players/001/179/19_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/it.png,2019
27,162835,https://sofifa.com/player/162835/samir-handano...,S. Handanovič,Samir Handanovič,GK,88,88,30000000.0,110000.0,33,1984-07-14,193,92,44.0,Inter,Italian Serie A,1.0,GK,1.0,NaN,2012-07-01,2021.0,

In [15]:
df[df.nation_position.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,177003,https://sofifa.com/player/177003/luka-modric/1...,L. Modrić,Luka Modrić,"CM, CDM",91,91,67000000.0,425000.0,32,1985-09-09,172,66,243.0,Real Madrid CF,Spain Primera Division,1.0,RCM,10.0,NaN,2012-08-01,2020.0,10,Croatia,NaN,NaN,NaN,Right,4,4,4,High/High,Lean (170-185),Yes,137400000.0,"#Dribbler, #Playmaker, #Engine, #Crosser, #Acr...","Leadership, Long Passer (AI), Long Shot Taker ...",76.0,76.0,90.0,91.0,70.0,67.0,86,72,55,93,76,90,85,78,88,93,80,72,93,90,94,79,68,89,58,82,62,83,79,92,82,84,60,76,73,13,9,7,14,9,NaN,78+3,78+3,78+3,86,85,85,85,86,88+3,88+3,88+3,87+3,88+3,88+3,88+3,87+3,83+3,82+3,82+3,82+3,83+3,80+3,71+3,71+3,71+3,80+3,20+3,https://cdn.sofifa.net/players/177/003/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/hr.png,2019
17,168542,https://sofifa.com/player/168542/david-josue-j...,David Silva,David Josué Jiménez Silva,"CAM, CM",89,89,51500000.0,275000.0,32,1986-01-08,173,67,10.0,Manchester City,English Premier League,1.0,LCM,21.0,NaN,2010-07-14,2020.0,45,Spain,NaN,NaN,NaN,Left,2,4,4,High/Medium,Normal (170-185),Yes,95300000.0,"#Dribbler, #Playmaker, #Acrobat","Avoids Using Weaker Foot, Finesse Shot, Flair,...",66.0,74.0,87.0,90.0,50.0,60.0,83,73,54,92,80,89,82,76,81,93,69,64,92,87,90,71,64,78,52,73,56,48,88,92,75,92,57,50,29,6,15,7,6,12,NaN,77+3,77+3,77+3,84,84,84,84,84,87+2,87+2,87+2,84+3,84+3,84+3,84+3,84+3,68+3,69+3,69+3,69+3,68+3,64+3,56+3,56+3,56+3,64+3,18+3,https://cdn.sofifa.net/players/168/542/19_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/es.png,2019
26,1179,https://sofifa.com/player/1179/gianluigi-buffo...,G. Buffon,Gianluigi Buffon,GK,88,88,4000000.0,80000.0,40,1978-01-28,192,92,73.0,Paris Saint-Germain,French Ligue 1,1.0,GK,1.0,NaN,2018-07-06,2020.0,27,Italy,NaN,NaN,NaN,Right,2,1,4,Medium/Medium,Normal (185+),Yes,7400000.0,NaN,"Team Player, Cautious With Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,15,13,37,17,26,20,13,35,28,49,43,55,79,49,39,75,39,69,13,38,28,12,50,22,70,13,11,11,88,87,74,90,83,46.0,31+3,31+3,31+3,32,33,33,33,32,35+3,35+3,35+3,34+3,35+3,35+3,35+3,34+3,30+3,33+3,33+3,33+3,30+3,28+3,29+3,29+3,29+3,28+3,85+3,https://cdn.sofifa.net/players/001/179/19_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/it.png,2019
27,162835,https://sofifa.com/player/162835/samir-handano...,S. Handanovič,Samir Handanovič,GK,88,88,30000000.0,110000.0,33,1984-07-14,193,92,44.0,Inter,Italian Serie A,1.0,GK,1.0,NaN,2012-07-01,2021.0,

`Takeaways:`
- Number of Nulls missing is greater than %90 so we will be eliminating these columns.

***

# `pace`, `shooting`,`dribbling`, `passing`, `defending`, `physic`.

In [16]:
df[df.pace.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
8,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,GK,91,93,72000000.0,250000.0,27,1990-11-07,193,76,11.0,Manchester United,English Premier League,1.0,GK,1.0,NaN,2011-07-01,2019.0,45,Spain,1362.0,GK,1.0,Right,3,1,4,Medium/Medium,Lean (185+),Yes,138600000.0,NaN,"GK Long Throw, Saves with Feet",NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,58.0,33+2,33+2,33+2,37-1,38-1,38-1,38-1,37-1,43+2,43+2,43+2,40+2,45+2,45+2,45+2,40+2,36+2,41+2,41+2,41+2,36+2,35+2,34+2,34+2,34+2,35+2,89+2,https://cdn.sofifa.net/players/193/080/19_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2019
9,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,44000000.0,130000.0,32,1986-03-27,193,92,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2021.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Normal (185+),Yes,72600000.0,NaN,"GK Long Throw, 1-on-1 Rush, Rushes Out Of Goal...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,55,11,30,14,11,59,48,54,60,51,84,35,25,77,43,80,16,29,30,12,70,47,70,17,10,11,91,88,91,88,88,57.0,35+4,35+4,35+4,40,41,41,41,40,46+4,46+4,46+4,43+4,48+4,48+4,48+4,43+4,36+4,42+4,42+4,42+4,36+4,34+4,34+4,34+4,34+4,34+4,88+2,https://cdn.sofifa.net/players/167/495/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2019
13,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,90,91,62000000.0,250000.0,26,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,SUB,25.0,NaN,2018-08-09,2024.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medium/Medium,Unique,Yes,131800000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66,20,18,16,87,91,74,87,88,49.0,29+3,29+3,29+3,29,30,30,30,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2019
14,200389,https://sofifa.com/player/200389/jan-oblak/190002,J. Oblak,Jan Oblak,GK,90,93,68000000.0,90000.0,25,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,N

In [17]:
df[df.shooting.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
8,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,GK,91,93,72000000.0,250000.0,27,1990-11-07,193,76,11.0,Manchester United,English Premier League,1.0,GK,1.0,NaN,2011-07-01,2019.0,45,Spain,1362.0,GK,1.0,Right,3,1,4,Medium/Medium,Lean (185+),Yes,138600000.0,NaN,"GK Long Throw, Saves with Feet",NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,58.0,33+2,33+2,33+2,37-1,38-1,38-1,38-1,37-1,43+2,43+2,43+2,40+2,45+2,45+2,45+2,40+2,36+2,41+2,41+2,41+2,36+2,35+2,34+2,34+2,34+2,35+2,89+2,https://cdn.sofifa.net/players/193/080/19_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2019
9,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,44000000.0,130000.0,32,1986-03-27,193,92,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2021.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Normal (185+),Yes,72600000.0,NaN,"GK Long Throw, 1-on-1 Rush, Rushes Out Of Goal...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,55,11,30,14,11,59,48,54,60,51,84,35,25,77,43,80,16,29,30,12,70,47,70,17,10,11,91,88,91,88,88,57.0,35+4,35+4,35+4,40,41,41,41,40,46+4,46+4,46+4,43+4,48+4,48+4,48+4,43+4,36+4,42+4,42+4,42+4,36+4,34+4,34+4,34+4,34+4,34+4,88+2,https://cdn.sofifa.net/players/167/495/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2019
13,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,90,91,62000000.0,250000.0,26,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,SUB,25.0,NaN,2018-08-09,2024.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medium/Medium,Unique,Yes,131800000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66,20,18,16,87,91,74,87,88,49.0,29+3,29+3,29+3,29,30,30,30,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2019
14,200389,https://sofifa.com/player/200389/jan-oblak/190002,J. Oblak,Jan Oblak,GK,90,93,68000000.0,90000.0,25,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,N

In [18]:
df[df.passing.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
8,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,GK,91,93,72000000.0,250000.0,27,1990-11-07,193,76,11.0,Manchester United,English Premier League,1.0,GK,1.0,NaN,2011-07-01,2019.0,45,Spain,1362.0,GK,1.0,Right,3,1,4,Medium/Medium,Lean (185+),Yes,138600000.0,NaN,"GK Long Throw, Saves with Feet",NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,58.0,33+2,33+2,33+2,37-1,38-1,38-1,38-1,37-1,43+2,43+2,43+2,40+2,45+2,45+2,45+2,40+2,36+2,41+2,41+2,41+2,36+2,35+2,34+2,34+2,34+2,35+2,89+2,https://cdn.sofifa.net/players/193/080/19_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2019
9,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,44000000.0,130000.0,32,1986-03-27,193,92,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2021.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Normal (185+),Yes,72600000.0,NaN,"GK Long Throw, 1-on-1 Rush, Rushes Out Of Goal...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,55,11,30,14,11,59,48,54,60,51,84,35,25,77,43,80,16,29,30,12,70,47,70,17,10,11,91,88,91,88,88,57.0,35+4,35+4,35+4,40,41,41,41,40,46+4,46+4,46+4,43+4,48+4,48+4,48+4,43+4,36+4,42+4,42+4,42+4,36+4,34+4,34+4,34+4,34+4,34+4,88+2,https://cdn.sofifa.net/players/167/495/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2019
13,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,90,91,62000000.0,250000.0,26,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,SUB,25.0,NaN,2018-08-09,2024.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medium/Medium,Unique,Yes,131800000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66,20,18,16,87,91,74,87,88,49.0,29+3,29+3,29+3,29,30,30,30,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2019
14,200389,https://sofifa.com/player/200389/jan-oblak/190002,J. Oblak,Jan Oblak,GK,90,93,68000000.0,90000.0,25,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,N

In [19]:
df[df.dribbling.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
8,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,GK,91,93,72000000.0,250000.0,27,1990-11-07,193,76,11.0,Manchester United,English Premier League,1.0,GK,1.0,NaN,2011-07-01,2019.0,45,Spain,1362.0,GK,1.0,Right,3,1,4,Medium/Medium,Lean (185+),Yes,138600000.0,NaN,"GK Long Throw, Saves with Feet",NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,58.0,33+2,33+2,33+2,37-1,38-1,38-1,38-1,37-1,43+2,43+2,43+2,40+2,45+2,45+2,45+2,40+2,36+2,41+2,41+2,41+2,36+2,35+2,34+2,34+2,34+2,35+2,89+2,https://cdn.sofifa.net/players/193/080/19_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2019
9,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,44000000.0,130000.0,32,1986-03-27,193,92,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2021.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Normal (185+),Yes,72600000.0,NaN,"GK Long Throw, 1-on-1 Rush, Rushes Out Of Goal...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,55,11,30,14,11,59,48,54,60,51,84,35,25,77,43,80,16,29,30,12,70,47,70,17,10,11,91,88,91,88,88,57.0,35+4,35+4,35+4,40,41,41,41,40,46+4,46+4,46+4,43+4,48+4,48+4,48+4,43+4,36+4,42+4,42+4,42+4,36+4,34+4,34+4,34+4,34+4,34+4,88+2,https://cdn.sofifa.net/players/167/495/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2019
13,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,90,91,62000000.0,250000.0,26,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,SUB,25.0,NaN,2018-08-09,2024.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medium/Medium,Unique,Yes,131800000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66,20,18,16,87,91,74,87,88,49.0,29+3,29+3,29+3,29,30,30,30,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2019
14,200389,https://sofifa.com/player/200389/jan-oblak/190002,J. Oblak,Jan Oblak,GK,90,93,68000000.0,90000.0,25,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,N

In [20]:
df[df.defending.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
8,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,GK,91,93,72000000.0,250000.0,27,1990-11-07,193,76,11.0,Manchester United,English Premier League,1.0,GK,1.0,NaN,2011-07-01,2019.0,45,Spain,1362.0,GK,1.0,Right,3,1,4,Medium/Medium,Lean (185+),Yes,138600000.0,NaN,"GK Long Throw, Saves with Feet",NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,58.0,33+2,33+2,33+2,37-1,38-1,38-1,38-1,37-1,43+2,43+2,43+2,40+2,45+2,45+2,45+2,40+2,36+2,41+2,41+2,41+2,36+2,35+2,34+2,34+2,34+2,35+2,89+2,https://cdn.sofifa.net/players/193/080/19_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2019
9,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,44000000.0,130000.0,32,1986-03-27,193,92,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2021.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Normal (185+),Yes,72600000.0,NaN,"GK Long Throw, 1-on-1 Rush, Rushes Out Of Goal...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,55,11,30,14,11,59,48,54,60,51,84,35,25,77,43,80,16,29,30,12,70,47,70,17,10,11,91,88,91,88,88,57.0,35+4,35+4,35+4,40,41,41,41,40,46+4,46+4,46+4,43+4,48+4,48+4,48+4,43+4,36+4,42+4,42+4,42+4,36+4,34+4,34+4,34+4,34+4,34+4,88+2,https://cdn.sofifa.net/players/167/495/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2019
13,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,90,91,62000000.0,250000.0,26,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,SUB,25.0,NaN,2018-08-09,2024.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medium/Medium,Unique,Yes,131800000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66,20,18,16,87,91,74,87,88,49.0,29+3,29+3,29+3,29,30,30,30,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2019
14,200389,https://sofifa.com/player/200389/jan-oblak/190002,J. Oblak,Jan Oblak,GK,90,93,68000000.0,90000.0,25,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,N

In [21]:
df[df.physic.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
8,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,GK,91,93,72000000.0,250000.0,27,1990-11-07,193,76,11.0,Manchester United,English Premier League,1.0,GK,1.0,NaN,2011-07-01,2019.0,45,Spain,1362.0,GK,1.0,Right,3,1,4,Medium/Medium,Lean (185+),Yes,138600000.0,NaN,"GK Long Throw, Saves with Feet",NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,58.0,33+2,33+2,33+2,37-1,38-1,38-1,38-1,37-1,43+2,43+2,43+2,40+2,45+2,45+2,45+2,40+2,36+2,41+2,41+2,41+2,36+2,35+2,34+2,34+2,34+2,35+2,89+2,https://cdn.sofifa.net/players/193/080/19_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2019
9,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,44000000.0,130000.0,32,1986-03-27,193,92,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2021.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Normal (185+),Yes,72600000.0,NaN,"GK Long Throw, 1-on-1 Rush, Rushes Out Of Goal...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,55,11,30,14,11,59,48,54,60,51,84,35,25,77,43,80,16,29,30,12,70,47,70,17,10,11,91,88,91,88,88,57.0,35+4,35+4,35+4,40,41,41,41,40,46+4,46+4,46+4,43+4,48+4,48+4,48+4,43+4,36+4,42+4,42+4,42+4,36+4,34+4,34+4,34+4,34+4,34+4,88+2,https://cdn.sofifa.net/players/167/495/19_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2019
13,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,90,91,62000000.0,250000.0,26,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,SUB,25.0,NaN,2018-08-09,2024.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medium/Medium,Unique,Yes,131800000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,14,14,13,33,12,13,19,20,35,23,46,52,61,84,45,36,68,38,70,17,23,15,13,44,27,66,20,18,16,87,91,74,87,88,49.0,29+3,29+3,29+3,29,30,30,30,29,32+3,32+3,32+3,31+3,32+3,32+3,32+3,31+3,29+3,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/19_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2019
14,200389,https://sofifa.com/player/200389/jan-oblak/190002,J. Oblak,Jan Oblak,GK,90,93,68000000.0,90000.0,25,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,N

`Takeaways:`
- Most of these null values are for goal keepers and can be replaced with 0 because these skills are not a requirement for their career field.

***
# `Null Values` Summary
The following `columns` have null values:
- value_eur, wage_eur, club_team_id, club_name, league_name, league_level, club_position, club_jersey_number, club_loaned_from, club_joined, club_contract_valid_until, nation_team_id, nation_position, nation_jersey_number, release_clause_eur, player_tags, player_traits, pace, shooting, passing, dribbling, defending, physic, goalkeeping_speed, club_logo_url, club_flag_url, nation_logo_url.
To address these Null values we have done the following:
- removed players players with null values.
- Club_position, club_joined, & club_contract_valid_until all seem to have similar payers with null values and those rows have been removed.
- For nation_positionNumber & nation_team_id the number of nulls values is greater than %90 of the data so we will be eliminating these columns.
- For pace, shooting,dribbling, passing, defending, and physic. Most of these null values are for goal keepers and can be replaced with 0 because these skills are not a requirement for their career field.

***

# Import [Prepare.py](https://github.com/Jules-Ray-Kayla-Bert-Capstone-FiFA/fifa/blob/main/prepare.py) File.
`Summary:`
    This file was created with the purpose of cleaning, encoding, scaling, dropping, and adding collumns to our existing data. It is necessary to complete this preperation step before we start exploring and modeling our data.

In [22]:
#defining df as our imported prepped data.
df = prepare.prepped_data(df)
df

Before dropping nulls, 53635 rows, 111 cols
After dropping nulls. 49417 rows. 66 cols
After clening the data and adding encoded columns. 36744 rows. 79 cols


,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,league_name,nationality_id,nationality_name,club_position,position,club_joined,club_contract_valid_until,body_type,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,pace,shooting,passing,dribbling,defending,physical,crossing,finishing,heading_accuracy,short_passing,volleys,skill_dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,year,total_wage,age_bins,height_bins,weight_bins,year_joined,seniority,league_encoded,club_position_encoded,work_rate_encoded,preferred_foot_encoded,age_bins_encoded,weight_bins_encoded,body_type_encoded
0,20801,Cristiano Ronaldo,"ST, LW",94,94,77000000.0,400000.0,33,187,83,45.0,Juventus,Italian Serie A,38,Portugal,ST,Striker,2018-07-10,2022.0,Unique,Right,4,5,5,High/Low,90,93,81,89,35,79,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,28,31,23,7,11,15,14,11,0,2019,77400000.0,older,medium,average,2018,1,6,2,7,2,1,2,1
1,158023,L. Messi,"CF, RW, ST",94,94,110500000.0,575000.0,31,170,72,241.0,FC Barcelona,Spain Primera Division,52,Argentina,RW,Right Wing,2004-07-01,2021.0,Unique,Left,4,4,5,Medium/Medium,88,91,88,96,32,61,77,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,33,28,26,6,11,15,14,8,0,2019,111075000.0,older,short,average,2004,15,5,1,5,1,1,2,1
2,190871,Neymar,LW,92,93,118500000.0,300000.0,26,175,68,73.0,Paris Saint-Germain,French Ligue 1,54,Brazil,LW,Left Wing,2017-08-03,2022.0,Unique,Right,5,5,5,High/Medium,92,84,83,95,32,59,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,27,24,33,9,9,15,15,11,0,2019,118800000.0,younger,medium,slim,2017,2,4,3,8,2,2,1,1
3,155862,Sergio Ramos,CB,91,91,51000000.0,375000.0,32,184,82,243.0,Real Madrid CF,Spain Primera Division,45,Spain,LCB,Left Centre Back,2005-08-01,2020.0,Normal (170-185),Right,3,3,4,High/Medium,75,63,71,71,91,84,66,60,91,78,66,61,73,67,72,84,76,75,78,85,66,79,93,84,83,55,84,90,52,63,68,89,92,91,11,8,9,7,11,0,2019,51375000.0,older,medium,average,2005,14,5,8,8,2,1,2,2
4,176580,L. Suárez,ST,91,91,80000000.0,450000.0,31,182,86,241.0,FC Barcelona,Spain Primera Division,60,Uruguay,ST,Striker,2014-07-11,2021.0,Normal (170-185),Right,4,3,5,High/Medium,80,90,79,88,52,85,77,94,77,82,88,87,86,84,64,90,86,75,82,93,83,86,69,90,83,85,87,41,92,84,85,62,45,38,27,25,31,33,37,0,2019,80450000.0,older,medium,average,2014,5,5,2,8,2,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53625,212270,M. Foden,GK,45,59,70000.0,1000.0,20,180,80,631.0,Ross County FC,Scottish Premiership,42,Scotland,SUB,Substitute,2016-03-02,2017.0,Normal (170-185),Right,3,1,1,Medium/Medium,0,0,0,0,0,0,15,14,15,20,12,13,13,12,15,17,42,40,34,46,60,23,48,40,61,12,23,12,14,13,24,11,14,11,45,43,44,40,45,41,2017,71000.0,younger,medium,average,2016,1,22,12,5,2,2,2,2
53626,214725,S. Ramsbottom,GK,45,56,50000.0,500.0,20,185,80,1571.0,Galway United,Rep. Ireland Airtricity League,14,England,SUB,Substitute,2016-07-20,2022.0,Normal (185+),Right,2,1,1,Medium/Medium,0,0,0,0,0,0,12,13,14,12,15,11,12,12,11,11,44,40,38,35,30,21,60,38,40,15,22,11,11,18,21,11,13,11,48,41,45,44,47,42,2017,50500.0,younger,medium,average,2016,1,24,12,5,2,2,2,4
53627,223621,D. Mahon,GK,45,59,60000.0,500.0,18,183,75,305.0,Bohemian FC,Rep. Ireland Airtricity League,25,Republic of Ireland,SUB,Substitute,2015-01-03,2023.0,Normal (170-185),Right,2,1,1,Medium/Medium,0,0,0,0,0,0,13,13,18,31,12

***
# End

***